In [ ]:
!pip install -q transformers datasets peft

In [ ]:
from transformers import pipeline

ask_llm = pipeline(
    model="Qwen/Qwen2.5-3B-Instruct",
    device = "cuda"

)

print(ask_llm("Who is tony stark")[0]["generated_text"])
# Initial LLM

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="tonyst.json")

print(dataset)
# Load Dataset

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct")

def preprocess(sample):
    sample = sample["prompt"] + "\n" + sample["completion"]

    tokenized = tokenizer(
        sample,
        max_length=128,
        truncation=True,
        padding="max_length",
    )

    tokenized["labels"] = tokenized["input_ids"].copy()
    return {
    "input_ids": tokenized["input_ids"],
    "attention_mask": tokenized["attention_mask"],
    "labels": tokenized["input_ids"]
}

data = dataset['train'].map(
    preprocess
)

# Tokenize Data


In [5]:
print(data[0])


{'prompt': 'What leadership principles can be learned from Tony Stark’s transition post-Afghanistan?', 'completion': "Tony Stark's transformation after his captivity marked a shift from reckless innovation to responsible leadership. He began prioritizing ethical technology development, transparency, and global impact over profit. His focus on clean energy and dismantling Stark Industries’ weapons division reflected a deep sense of accountability.", 'input_ids': [3838, 11438, 16170, 646, 387, 9498, 504, 18528, 46749, 748, 9142, 1736, 6691, 69, 15708, 8944, 5267, 57734, 46749, 594, 17991, 1283, 806, 82919, 12864, 264, 6407, 504, 53217, 18770, 311, 8480, 11438, 13, 1260, 6009, 44677, 4849, 30208, 5440, 4401, 11, 27231, 11, 323, 3644, 5421, 916, 11372, 13, 5301, 5244, 389, 4240, 4802, 323, 48112, 2718, 46749, 36428, 527, 9893, 12804, 25911, 264, 5538, 5530, 315, 38142, 13, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643

In [6]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    'Qwen/Qwen2.5-3B-Instruct',
    device_map = "cuda",
    torch_dtype = torch.float16
)

lora_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    target_modules = ["q_proj", "k_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)

# Define LoRA Config

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    num_train_epochs=10,
    learning_rate=0.001,
    logging_steps=25,
    per_device_train_batch_size=1

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data
)


⚙️  Running in WANDB offline mode


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [9]:
trainer.train()
# Save Model

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
25,2.619300
50,1.529700
75,1.349600
100,1.371300
125,1.298000
150,1.355200
175,1.182900
200,1.121800
225,1.121000
250,1.015500


TrainOutput(global_step=1580, training_loss=0.506374632218216, metrics={'train_runtime': 380.6926, 'train_samples_per_second': 4.15, 'train_steps_per_second': 4.15, 'total_flos': 3370063264481280.0, 'train_loss': 0.506374632218216, 'epoch': 10.0})

In [ ]:

trainer.save_model("./my_qwen")
tokenizer.save_pretrained("./my_qwen")

# Save Model

In [11]:

ask_llm = pipeline(
    model="./my_qwen",
    tokenizer="./my_qwen",
    device="cuda",
    torch_dtype=torch.float16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


In [19]:

ask_llm("Suits of tony stark, name all along with properties")
# Inference Example

[{'generated_text': 'Suits of tony stark, name all along with properties.\n1. **Iron Man Mark I** - Armorated suit crafted from salvaged B.B. guns, it was the first prototype featuring a chest plate and arm cannons.\n2. **Iron Man Mark II** - Introduced the concept of a flight system and a more sophisticated armament system with shoulder-mounted projectors.\n3. **Iron Man Mark III** - Added horizontal flight wings and improved the armortech, making it the first fully functional production model.\n4. **Iron Man Mark V** - Introduced the iconic gold and iridium plating, and expanded the armament to include a variety of projectile weapons.\n5. **Iron Man Mark VI** - Features a more streamlined flight system and a secondary power source in the gauntlet for enhanced combat capabilities.\n6. **Iron Man Mark VII** - Featured a retractable wing system in the wings for more stealth and a high-energy laser in the hand for long-range combat.\n7. **Iron Man Mark VIII** - Introduced the concept of 

In [ ]:
!zip -r my_qwen.zip ./my_qwen
# Zipping model